<h1>Code Snippets for Parsing the Gigantic Stack Overflow Dataset</h1>

<p><b>Step1:</b> <u>Download the data dump from the official Stack Exchange <a href='https://archive.org/download/stackexchange'>archive</a> if you want to have a data dump version published around April 2024. If you want to download the latest version you need to follow the instruction <a href='https://stackoverflow.com/help/data-dumps'>here</a>.</u> It contains 8 XML files namely Badges, Comments, PostHistory, PostLinks, Posts, Tags, Users and Votes.</p> 

<p><b>Step 2: </b><u>Split the large sized XML files to smaller chunks</u>. Apart from PostLinks, and Tags, all other files are too large to process easily in one go. So the next step is to divide them into small chunks. For this purpose, <a href='https://git-scm.com/download/win'>git bash</a> can be used which provides split command to split files line by line or by size. We used the split by line option. After extracting each of the above mentioned compressed files, open git bash terminal in the respective root directory and type the following commands respectively. </p><br>
<ul>time split -l 50000 -d -a 5 Badges.xml Badges/Split/Badges</ul>
<ul>time split -l 50000 -d -a 5 Users.xml  Users/Split/Users</ul>
<ul>time split -l 50000 -d -a 5 Comments.xml  Comments/Split/Comments</ul>
<ul>time split -l 50000 -d -a 5 Votes.xml  Votes/Split/Votes</ul>
<ul>time split -l 50000 -d -a 5 Posts.xml  Posts/Split/Posts</ul>
<ul>time split -l 50000 -d -a 5 PostHistory.xml  PostHistory/Split/PostHistory</ul>


<p><b>Step 3: </b><u>Converting splitted files to proper XML files.</u> As the split operation was performed line by line, the first chunk contains XML header whereas the last chunk contains the closing tag of the respective file. The rest of the chunks lack proper XML header/footer. This Python script is used to add these missing XML header/footer tags to convert the chunks to proper XML files. After running the above command (e.g. <code>split -l 50000 -d -a 10 Posts.xml Posts/Split/Posts</code>, the <code>Posts/Split</code> folder will contain all the generated chunks so we need to provide thhe full path to this folder as <code>input_path</code>. We used the following code snippet to convert these chunks into small sized XML files. Replace the XML header/footer strings in the code snippet (e.g <code> posts </code> and <code> /posts </code>) based on the file that you are dealing with e.g.  users, votes, posthistory, badges or comments and change the input and output paths accordingly. </p>

In [ ]:
import glob
import os
import time

input_path = r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Posts\Split'
output_path = r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Posts\Split\\'

start_time = time.perf_counter()

# ---- Get files in sorted order ----
files = sorted(glob.glob(os.path.join(input_path, '*')))

total_files = len(files)

if total_files == 0:
    print("No input files found.")
    exit()

for idx, filename in enumerate(files):

    print(f"Reading file {filename}")

    # Determine first and last file
    is_first = (idx == 0)
    is_last = (idx == total_files - 1)

    head, tail = os.path.split(filename)
    outputfile = output_path + tail + '.xml'

    print(f"Writing file to {outputfile}")

    with open(filename, 'r', encoding='utf8') as f, \
         open(outputfile, "w", encoding='utf8') as output:

        # ---- Add opening tag (skip first file) ----
        if not is_first:
            output.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n<posts>")

        # ---- Stream file content (memory safe) ----
        for line in f:
            output.write(line)

        # ---- Add closing tag (skip last file) ----
        if not is_last:
            output.write("</posts>")

end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<p><b>Step 4: </b><u>Saving XML based data in database for further querying.</u> We used MS SQL Server Developer Edition to store the data as the Express Edition restricts the database size to 10GB at maximum. However, in our case that dataset was much larger than that. It is worth mentioning here that the Developer Edition cannot be used in production environment. We further used SQL Server Management Studio (SSMS) to easily create and manipulate databases using a GUI. Both <a href='https://www.microsoft.com/en-us/sql-server/sql-server-downloads?msockid=236f7f3cd0c269f836b36be8d4c26fdb'>SQL Server </a> and <a href='https://learn.microsoft.com/en-us/sql/ssms/download-sql-server-management-studio-ssms?view=sql-server-ver16'> SSMS </a> are available for use freely by Microsoft. The default location used by SQL Server to store database files on a Windows based system is <code>C:\Program Files\Microsoft SQL Server\MSSQL16.MSSQLSERVER\MSSQL\DATA\</code>. Using <code>New Database</code> wizard in SSMS, create new database and change the default location for storing the dataabase files to some other directory as your <code>C:</code> drive may have limited space. Instead of storing all data in a single database, we created separate databases corresponding to each of the above mentioned XML files (i.e. <code> StackOverflowBadgesDb_Jan2026, StackOverflowCommentsDb_Jan2026, StackOverflowPostHistoryDb_Jan2026, StackOverflowPostLinksDb_Jan2026, StackOverflowPostsDb_Jan2026, StackOverflowTagsDb_Jan2026, StackOverflowUsersDb_Jan2026, StackOverflowVotesDb_Jan2026 </code> respectively.)</p>

<p><b>Step 5: </b><u>Creating Tables for Storing Data</u>. After creating the respective databases, the next step is to create tables in which the actual data will be stored from the XML files. We used the following SQL queries to create corresponding tables. </p>


<p><b>Step 6: </b><u>Reading XML files and inserting records in respective database tables</u>. Now we need to read the small sized XML files one by one and store the data in the respective tables created above. For this purpose, the code snippet used for parsing each of the 8 XML file types namely Badges, Comments, PostHistory, PostLinks, Posts, Tags, Users, and Votes is given below. Before using these snippets, replace the server and database names and the path of directory containing splitted XML files. </p>

<h2>Badges Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os

server = 'localhost'
database = 'StackOverflowBadgesDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

badges_xml_path =r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Badges\SplitXML'

for filename in glob.glob(os.path.join(badges_xml_path, '*')):

    print(filename)
    
    badges_df = pd.read_xml(filename)
    badges_df = badges_df.fillna(0)

    # print (badges_df.head())
    # print('*************')
 
    query = "insert into StackOverflowBadges (Id, UserId, Name, Date, Class,TagBased) values (?,?,?,?,?,?)"

    cursor.executemany(query, badges_df.values.tolist())
    cursor.commit()
    print("Added in database")
    batch_time = time.perf_counter()
    print(f"\nTime taken till now {batch_time - start_time:.2f} seconds")
end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<h2>Comments Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os
server = 'localhost'
database = 'StackOverflowCommentsDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

comments_xml_path =r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Comments\SplitXML'

for filename in glob.glob(os.path.join(comments_xml_path, '*')):

    print(filename)
    comments_df = pd.read_xml(filename)
    comments_df = comments_df.fillna(0)

    # print (comments_df.head())
    # print('*************')
    columnnames= list(comments_df.columns.values)
    columns=','.join(columnnames)
    # query="insert into StackOverflowComments ("+columns+") values (?,?,?,?,?,?,?,?)"
    placeholders = ",".join(["?"] * len(columnnames))

    query = f"insert into StackOverflowComments ({columns}) VALUES ({placeholders})"
    
    cursor.executemany(query, comments_df.values.tolist())
    cursor.commit()
    print("Added in database")
    batch_time = time.perf_counter()
    print(f"\nTime taken till now {batch_time - start_time:.2f} seconds")

end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<h2>PostHistory Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os
import time


server = 'localhost'
database = 'StackOverflowPostHistoryDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()
post_history_xml_path ='E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\PostHistory\SplitXML\\'

for filename in glob.glob(os.path.join(post_history_xml_path, '*')):

    print(filename)
    PostHistory_df = pd.read_xml(filename)
    
    PostHistory_df =  PostHistory_df.fillna(value={'Id':0,'PostHistoryTypeId':0,'PostId':0,'RevisionGUID':0,'CreationDate':0,'UserId':0,'Text':'','ContentLicense':'','UserDisplayName':'', 'Comment':''})
    # print (PostHistory_df.head())
    # print('*************')
    columnnames= list(PostHistory_df.columns.values)
    columns=','.join(columnnames)
    query="insert into StackOverflowPostHistory ("+columns+") values (?,?,?,?,?,?,?,?,?,?)"
    
    # query = "insert into StackOverflowPostHistory(Id,PostHistoryTypeId,PostId,RevisionGUID,CreationDate,UserId,Text,ContentLicense,Comment,UserDisplayName) values (?,?,?,?,?,?,?,?,?,?)"

    cursor.executemany(query, PostHistory_df.values.tolist())
    cursor.commit()
    print("Added in database")
    batch_time = time.perf_counter()
    print(f"\nTime taken till now {batch_time - start_time:.2f} seconds")
end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<h2>PostLinks Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc

server = 'localhost'
database = 'StackOverflowPostLinksDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

PostLinks_xml_path =r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\PostLinks.xml'
PostLinks_df = pd.read_xml(PostLinks_xml_path)
PostLinks_df = PostLinks_df.fillna(0)

print (PostLinks_df.head())
print('*************')
 
query = "insert into StackOverflowPostLinks (Id,CreationDate,PostId,RelatedPostId,LinkTypeId) values (?,?,?,?,?)"

cursor.executemany(query, PostLinks_df.values.tolist())
cursor.commit()
print("Added in database")
end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<h2>Posts Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os
import re
from datetime import datetime
import traceback
import time

server = 'localhost'
database = 'StackOverflowPostsDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()
posts_xml_path =r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Posts\SplitXML'

for filename in glob.glob(os.path.join(posts_xml_path, '*')):

    print(filename)
    posts_df = pd.read_xml(filename)
    # print(posts_df.head())
    posts_df =  posts_df.fillna(value={'Id':0,'PostTypeId':0,'AcceptedAnswerId':0,'CreationDate':'','Score':0,'ViewCount':0,'OwnerUserId':0, 'LastEditorUserId':0, 'LastEditDate':0, 'LastActivityDate':0,'AnswerCount':0,'CommentCount':0, 'ParentId':0, 'CommunityOwnedDate':0 ,  'ClosedDate':0,
                            'FavoriteCount':0})
    posts_df['Id']=posts_df['Id'].astype(int)
    posts_df['PostTypeId']=posts_df['PostTypeId'].astype(int)
    if 'AcceptedAnswerId' in posts_df.columns:
        posts_df['AcceptedAnswerId']=posts_df['AcceptedAnswerId'].astype(int)
    posts_df['CreationDate']= pd.to_datetime(posts_df['CreationDate'], format="mixed")
    posts_df['Score']=posts_df['Score'].astype(int)
    if 'ViewCount' in posts_df.columns:
        posts_df['ViewCount']=posts_df['ViewCount'].astype(int)
    posts_df['OwnerUserId']=posts_df['OwnerUserId'].astype(int)
    posts_df['LastEditorUserId']=posts_df['LastEditorUserId'].astype(int)
    posts_df['LastEditDate']= pd.to_datetime(posts_df['LastEditDate'], format="mixed")
    posts_df['LastActivityDate']= pd.to_datetime(posts_df['LastActivityDate'], format="mixed")
    if 'AnswerCount' in posts_df.columns:
        posts_df['AnswerCount']=posts_df['AnswerCount'].astype(int)
    if 'CommentCount' in posts_df.columns:
        posts_df['CommentCount']=posts_df['CommentCount'].astype(int)
    if 'ParentId' in posts_df.columns:
        posts_df['ParentId']=posts_df['ParentId'].astype(int)
    if 'CommunityOwnedDate' in posts_df.columns:
        posts_df['CommunityOwnedDate']= pd.to_datetime(posts_df['CommunityOwnedDate'], format="mixed")
    if 'ClosedDate' in posts_df.columns:
        posts_df['ClosedDate']= pd.to_datetime(posts_df['ClosedDate'], format="mixed")

    if 'FavoriteCount' in posts_df.columns:
        posts_df['FavoriteCount']=posts_df['FavoriteCount'].astype(int)
    columnnames= list(posts_df.columns.values)
    columns=','.join(columnnames)
    placeholders = ",".join(["?"] * len(columnnames))

    query = f"insert into StackOverflowPosts ({columns}) VALUES ({placeholders})"

    posts_df = posts_df.fillna('')

    cursor.executemany(query, posts_df.values.tolist())
    cursor.commit()
    print("Added in database")
    batch_time = time.perf_counter()
    print(f"\nTime taken till now {batch_time - start_time:.2f} seconds")
end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<h2>Tags Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc

server = 'localhost'
database = 'StackOverflowTagsDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

row_list = []
errorcount = 0
tags_xml_path =r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Tags.xml'
tags_df = pd.read_xml(tags_xml_path)
tags_df = tags_df.fillna('')

print (tags_df.head())
print('*************')

query = "insert into StackOverflowTags (Id, TagName, TagCount ,ExcerptPostId ,WikiPostId) values (?,?,?,?,?)"

cursor.executemany(query, tags_df.values.tolist())
cursor.commit()
print("Added in database")
end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<h2>Users Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os

server = 'localhost'
database = 'StackOverflowUsersDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

users_xml_path =r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Users\SplitXML'

for filename in glob.glob(os.path.join(users_xml_path, '*')):
    print(filename)
    users_df = pd.read_xml(filename)
    users_df = users_df.fillna(0)
    
    # print (users_df.head())
    # print(users_df.columns)
    # print('*************')
    columnnames= list(users_df.columns.values)
    columns=','.join(columnnames)
#     print(columnnames)
    query="insert into StackOverflowUsers ("+columns+") values (?,?,?,?,?,?,?,?,?,?,?,?)"
#     query = "insert into StackOverflowUsers ( Id, Reputation, CreationDate, DisplayName, LastAccessDate, Views, UpVotes, DownVotes, AccountId, WebsiteUrl, AboutMe,ProfileImageUrl, Location) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

    cursor.executemany(query, users_df.values.tolist())
    cursor.commit()
    print("Added in database")
    batch_time = time.perf_counter()
    print(f"\nTime taken till now {batch_time - start_time:.2f} seconds")

end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")

<h2>Votes Parser for Stack Overflow Dataset</h2>

In [ ]:
import pandas as pd
import pyodbc
import glob
import os
import time 
server = 'localhost'
database = 'StackOverflowVotesDb_Jan2026' 
start_time = time.perf_counter()
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()

votes_xml_path =r'E:\Dataset_2026\StackOverflow\Jan2026\stackoverflow.com\Votes\SplitXML'

for filename in glob.glob(os.path.join(votes_xml_path, '*')):

    print(filename)
    votes_df = pd.read_xml(filename)
    votes_df = votes_df.fillna(0)

    # print (votes_df.head())
    # print('*************')
    columnnames= list(votes_df.columns.values)
    columns=','.join(columnnames)
    if 'UserId' in votes_df.columns and 'BountyAmount' in votes_df.columns :
        votes_df['UserId']=votes_df['UserId'].astype(int)
        votes_df['BountyAmount']=votes_df['BountyAmount'].astype(int)
        
        query="insert into StackOverflowVotes ("+columns+") values (?,?,?,?,?,?)"
    else:
        query = "insert into StackOverflowVotes (Id, PostId,VoteTypeId,CreationDate) values (?,?,?,?)"
        
    cursor.executemany(query, votes_df.values.tolist())
    cursor.commit()
    print("Added in database")
    batch_time = time.perf_counter()
    print(f"\nTime taken till now {batch_time - start_time:.2f} seconds")
end_time = time.perf_counter()
print(f"\nCompleted in {end_time - start_time:.2f} seconds")